In [1]:
from IPython.display import display
from collections import namedtuple
from datasets import load_dataset
from huggingface_hub import login
from jaxtyping import Int, Float
from jinja2 import Environment, PackageLoader, select_autoescape
from pathlib import Path
from pprint import pp
from torch import Tensor
from tqdm import tqdm
from transformers import LlamaForCausalLM, LlamaTokenizer
from utils.truthful_qa_ds import get_question_answer_dataset
import copy
import numpy as np
import torch
import torch
import torch.nn as nn
import torch.nn.functional as F
import transformers

login()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pp(device)
np_rand = np.random.default_rng(seed=100500)
model_type = torch.float16

device(type='cuda')


In [2]:
# Load model
# llama_path = "../llama/7bf_converted/"
tokenizer = LlamaTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
tokenizer.add_special_tokens({"pad_token": "<pad>"})
# tokenizer.pad_token = tokenizer.eos_token

model = LlamaForCausalLM.from_pretrained(
    "meta-llama/Llama-2-7b-chat-hf",
    torch_dtype=model_type,
    device_map=device,
)
model.eval()
pp(model)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_

In [3]:
true_token = tokenizer.encode("True")[1]
false_token = tokenizer.encode("False")[1]
print(true_token)
print(false_token)

5852
7700


In [4]:
truthfulqa = load_dataset("truthful_qa", "generation")  # 817 rows
env = Environment(loader=PackageLoader("utils"), autoescape=select_autoescape())

Generating validation split:   0%|          | 0/817 [00:00<?, ? examples/s]

In [5]:
# Accuracy on the TruthfulQA dataset, few shot:
count = 0
correct_samples = []
correct_n = 0
qa_t = env.get_template("question_answer.jinja")
with torch.no_grad():
    p_bar = tqdm(list(enumerate(truthfulqa["validation"])))
    for i, row in p_bar:
        def is_correct_answer(take_correct):
            input_ = qa_t.render(
                row,
                is_correct_answer=take_correct
            ),
            t_output = tokenizer(input_, return_tensors="pt")
            t_output = {k: t_output[k].to(device) for k in t_output}
            outputs = model(**t_output, output_hidden_states=False)
            pred = outputs.logits[0, -1].softmax(dim=-1)
            predicted = (pred[true_token] > pred[false_token]).item()
            return predicted == take_correct
        with_true = is_correct_answer(True)
        count += 1
        if with_true:
            correct_n += 1
        with_false = is_correct_answer(False)
        count += 1
        if with_false:
            correct_n += 1
        if with_true and with_false:
            correct_samples.append(row)
        p_bar.set_description(
            f"Correct {correct_n}, count {count}, accuracy {correct_n / count:.4}, both {len(correct_samples)}"
        )
# Result: Correct 972, count 1634, accuracy 0.5949, both 261

Correct 969, count 1634, accuracy 0.593, both 259: 100%|██████████| 817/817 [01:53<00:00,  7.21it/s] 


In [7]:
# Accuracy on the TruthfulQA dataset, few shot:
count = 0
correct_samples = []
correct_n = 0
qa_t = env.get_template("question_answer.jinja")
with torch.no_grad():
    p_bar = tqdm(list(enumerate(truthfulqa["validation"])))
    for i, row in p_bar:
        def is_correct_answer(take_correct):
            input_ = qa_t.render(
                row,
                is_correct_answer=take_correct
            ),
            t_output = tokenizer(input_, return_tensors="pt")
            t_output = {k: t_output[k].to(device) for k in t_output}
            outputs = model(**t_output, output_hidden_states=False)
            pred = outputs.logits[0, -1].softmax(dim=-1)
            predicted = (pred[true_token] > pred[false_token]).item()
            return predicted == take_correct
        with_true = is_correct_answer(True)
        count += 1
        if with_true:
            correct_n += 1
        with_false = is_correct_answer(False)
        count += 1
        if with_false:
            correct_n += 1
        if with_true and with_false:
            correct_samples.append(row)
        p_bar.set_description(
            f"Correct {correct_n}, count {count}, accuracy {correct_n / count:.4}, both {len(correct_samples)}"
        )
# Result: Correct 972, count 1634, accuracy 0.5949, both 261

Correct 950, count 1634, accuracy 0.5814, both 242: 100%|██████████| 817/817 [02:03<00:00,  6.62it/s]


In [8]:
# Now calculate accuracy for compound sentences for correctly 
# detected samples. Expectation: should guess all correctly.

count = 0
correct_compound = []
correct_n = 0
qas_t = env.get_template("question_answers.jinja")
with torch.no_grad():
    p_bar = tqdm(list(enumerate(correct_samples)))
    for i, row in p_bar:
        def is_correct_answer(take_correct):
            input_ = qas_t.render(
                row,
                is_correct_answer=take_correct,
                is_disjunction=False,
            ),
            t_output = tokenizer(input_, return_tensors="pt")
            t_output = {k: t_output[k].to(device) for k in t_output}
            outputs = model(**t_output, output_hidden_states=False)
            pred = outputs.logits[0, -1].softmax(dim=-1)
            predicted = (pred[true_token] > pred[false_token]).item()
            return predicted == take_correct
        with_true = is_correct_answer(True)
        count += 1
        if with_true:
            correct_n += 1
        with_false = is_correct_answer(False)
        count += 1
        if with_false:
            correct_n += 1
        if with_true and with_false:
            correct_compound.append(row)
        p_bar.set_description(
            f"Correct {correct_n}, count {count}, accuracy {correct_n / count:.4}, both {len(correct_compound)}"
        )

# Random? Correct 289, count 522, accuracy 0.5536, both 48: 100%|██████████| 261/261 [00:44<00:00,  5.84it/s]

Correct 261, count 484, accuracy 0.5393, both 37: 100%|██████████| 242/242 [00:53<00:00,  4.49it/s]


In [9]:
# Accuracy on the TruthfulQA dataset, few shot:
count = 0
correct_samples = []
correct_n = 0
qa_t = env.get_template("question_answer.jinja")
with torch.no_grad():
    p_bar = tqdm(list(enumerate(truthfulqa["validation"])))
    for i, row in p_bar:
        def is_correct_answer(take_correct):
            input_ = qa_t.render(
                row,
                is_correct_answer=take_correct
            ),
            t_output = tokenizer(input_, return_tensors="pt")
            t_output = {k: t_output[k].to(device) for k in t_output}
            outputs = model(**t_output, output_hidden_states=False)
            pred = outputs.logits[0, -1].softmax(dim=-1)
            predicted = (pred[true_token] > pred[false_token]).item()
            return predicted == take_correct
        with_true = is_correct_answer(True)
        count += 1
        if with_true:
            correct_n += 1
        with_false = is_correct_answer(False)
        count += 1
        if with_false:
            correct_n += 1
        if with_true and with_false:
            correct_samples.append(row)
        p_bar.set_description(
            f"Correct {correct_n}, count {count}, accuracy {correct_n / count:.4}, both {len(correct_samples)}"
        )
# Result: Correct 972, count 1634, accuracy 0.5949, both 261

Correct 969, count 1634, accuracy 0.593, both 259: 100%|██████████| 817/817 [01:53<00:00,  7.22it/s] 


In [10]:
# Now calculate accuracy for compound sentences for correctly 
# detected samples. Expectation: should guess all correctly.

count = 0
correct_compound = []
correct_n = 0
qas_t = env.get_template("question_answers.jinja")
with torch.no_grad():
    p_bar = tqdm(list(enumerate(correct_samples)))
    for i, row in p_bar:
        def is_correct_answer(take_correct):
            input_ = qas_t.render(
                row,
                is_correct_answer=take_correct,
                is_disjunction=False,
            ),
            t_output = tokenizer(input_, return_tensors="pt")
            t_output = {k: t_output[k].to(device) for k in t_output}
            outputs = model(**t_output, output_hidden_states=False)
            pred = outputs.logits[0, -1].softmax(dim=-1)
            predicted = (pred[true_token] > pred[false_token]).item()
            return predicted == take_correct
        with_true = is_correct_answer(True)
        count += 1
        if with_true:
            correct_n += 1
        with_false = is_correct_answer(False)
        count += 1
        if with_false:
            correct_n += 1
        if with_true and with_false:
            correct_compound.append(row)
        p_bar.set_description(
            f"Correct {correct_n}, count {count}, accuracy {correct_n / count:.4}, both {len(correct_compound)}"
        )

# Random? Correct 289, count 522, accuracy 0.5536, both 48: 100%|██████████| 261/261 [00:44<00:00,  5.84it/s]

Correct 287, count 518, accuracy 0.5541, both 48: 100%|██████████| 259/259 [00:55<00:00,  4.64it/s]


In [11]:
# Previous 2 accuracy cells without: "Answer these questions:". It is 1.8 percent higher
# 